<a href="https://colab.research.google.com/github/Jeanne26/AIES/blob/main/aies2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clingo
!pip install clyngor-with-clingo


In [ ]:
import clingo
import tempfile
from clyngor import ASP, solve

In [ ]:
program1 = """

% FACTS

% Main protagonists of the story
person(p;f).

% Possible locations
location(madrid;cemetery;family).

% Current and potential situations
situation(s1).
%situation_initiale(s2).
%situation_intiale(s3).


% Belief base for the environment's objective facts and for the main agent's beliefs
plannumber(0..1).
beliefbase(env).
beliefbase(p(P)):-plannumber(P).


% Description of situation s1 (current situation)

%% Beliefs about location
believe(env, at(r,cemetery)) :-situation(s1).
believe(p(0), at(r,family)):-situation(s1).

%% Beliefs about hostility
believe(p(0),hostile(f)) :-situation(s1).
believe(env,hostile(f)):-situation(s1).
believe(env,murderer(f)):-situation(s1).
believe(p(0),murderer(f)):-situation(s1).

%% Beliefs about credible locations
believe(p(0),credible(at(r,madrid), f)):-situation(s1).
believe(p(0),credible(at(r,cemetery), f)):-situation(s1).
believe(p(0),credible(at(r,family), f)):-situation(s1).
believe(env,credible(at(r,cemetery), f)):-situation(s1).
believe(env,credible(at(r,family), f)):-situation(s1).

%Inertia
believe(p(P+1),F):- believe(p(P),F), not -believe(p(P+1),F), plannumber(P+1).

-believe(p(P+1), credible(at(r,X), p2)):- act(p(P),tell(p(P),at(r,X),f,N)), not believe(p(P), ask(f,where,p(P),N+1)), believe(env,ask(f,where,p(_),N+1)).


% Relation between questioning round and actions
%1{act(tell(p,at(r,L), f)):location(L);act(silence(p, f))}1.

nbAsk(0..1).
believe(env, ask(f,where,p,0)).
believe(p(0),ask(f,where,p,0)).

%anticipated reaction
1{act(tell(p(P),at(r,L),f,N)): location(L);act(silence(p(P),f))}1:- believe(p(P), ask(f,where,p,N)), believe(env,ask(f,where,p,N)), nbAsk(N).

%unanticipated reaction
1{act(tell(p(P+1),at(r,L),f,N)): location(L);act(silence(p(P+1),f))}1:- believe(env,ask(f,where,p,N)),not act(tell(p(P),_,f,N)),not believe(p(P),ask(f,where,p,N)), nbAsk(N), plannumber(P).



% ----------------------------------------------------------------------------------------------------------------

% RULES FOR PHYSICAL ACTIONS
% Rule for checking a location
believe(Y,check(P2,X,P1)) :- act(tell(P1, at(r,X), P2, N)), believe(Y,credible(at(r,X), P2)),nbAsk(N).

% Rule for reasking a question based on non-credible information
believe(Y,ask(P2, where, P1, N+1)) :- act(tell(P1, at(r,X), P2, N)), not believe(Y,credible(at(r,X), P2)), nbAsk(N),nbAsk(N+1), beliefbase(Y).

% Rule for evasion
believe(Y,evade(P1,P2)) :- believe(Y,check(P2,madrid,P1)).

% Rule for killing
believe(Y,kill(P2,r,P1)) :- believe(Y,check(P2,X,P1)), believe(Y,at(r,X)), believe(Y,hostile(P2)).

% Rule for helping
believe(Y,help(P2)) :- believe(Y,check(P2,X,P1)), believe(Y,at(r,X)), not believe(Y,hostile(P2)).

% Rule for agents loosing patience (first condition)
believe(Y,patienceLost(P2, P1)) :- act(tell(P1, at(r,X), P2, N)), not believe(Y,credible(at(r,X), P2)), believe(Y,hostile(P2)), nbAsk(N), not nbAsk(N+1).

% Rule for agents loosing patience (second condition)
believe(Y,patienceLost(P2, P1)) :- act(silence(P1, P2,N)), believe(Y,hostile(P2)),nbAsk(N).

% Rule for wrong information provided
believe(Y,wrongInfo(P2, P1)) :- believe(Y,check(P2, X, P1)), not believe(Y,at(r,X)).

% Rule for an agent being harmed based on another agent loosing patience
believe(Y,harm1(P2, P1)) :- believe(Y,patienceLost(P2, P1)).

% Rule for an agent being harmed based on hostility and another agent providing wrong information
believe(Y,harm2(P2, P1)) :- believe(Y,wrongInfo(P2, P1)), believe(Y,hostile(P2)), not believe(Y,evade(P1,P2)).


% ----------------------------------------------------------------------------------------------------------------

% RULES FOR SPEECH ACTS
% Rule for honest communication based on agents' telling what they believe
honest(P1,F) :- act(tell(P1,F,_)), believe(P1,F).

% Rule for dishonest communication based on agents' not telling what they believe
dishonest(P1,F) :- act(tell(P1,F,_)), not believe(P1,F).

% Rules for telling objective truth based on the environment's belief
truth(P1,F) :- act(tell(P1,F,_)), believe(env,F).

% Rules for telling objective falsity based on the environment's belief
falsity(P1,F) :- act(tell(P1,F,_)), not believe(env,F).

% Rules for telling an objective truth or an erroneous truth
objective_truth(P1,F) :- honest(P1,F), truth(P1,F).
erroneous_truth(P1,F) :- honest(P1,F), falsity(P1,F).

% Rules for telling an objective lie onAlternative  1,  Pablo  once  again  lies  to  the  hostile842Falangists  by  claiming  that  Ramon  is  in  Madrid.  Here,843unlike  in  the  original  version  of  the  story,  the  lie  helps844both  Ramon  and  Pablo  since  the  Falangists  will  search845for  Ramon  in  distant  Madrid,  giving  Pablo  enough  time846to  escape.  But  despite  its  greater  advantage,  “Madrid”847is  considered  equallypermissibleas  “Cemetery”  under848principialism.  Inprincipialism1,  the  exception  al-849lowing a lie is granted due to the hostility of the Falangists.850Underprincipialism2,  counterfactual  reasoning  shows851that  telling  the  truth  (“Ramon  is  at  his  family’s  house”)852would  lead  to  worse  outcomes  than  either  lie,  rendering853“Madrid”  and  “Cemetery”  morally  equivalent  within  the854framework  of  this  theory.  This  equivalence  does  not  hold855underconsequentialism1. From the perspective of ex-856pected utility, “Madrid” uniquely enables both Ramon’s and857Pablo’s survival, whereas “Cemetery”, though still protec-858tive of Ramon, increases the personal risk to Pablo. Thus,859whileprincipialismdeems both theobjectivelie860“Madrid”  and  theerroneouslie“Cemetery”  equally861permissible,consequentialism1identifies  “Madrid”  as862the uniquely optimal choicer an erroneous lie
objective_lie(P1,F) :- dishonest(P1,F), falsity(P1,F).
erroneous_lie(P1,F) :- dishonest(P1,F), truth(P1,F).


%%%%%%%%%%%%%%%  DEONTOLOGISM %%%%%%%%%%%%%%%%%%

% Maxims prohibiting lying and murder for deontologism
maxim(doNotLie;doNotEnableMurder).

% Rule concerning the violation of the maxim against lying
violated(doNotLie,P1) :- dishonest(P1,F).

% Rule concerning the violation of the maxim against murder
violated(doNotEnableMurder,P1) :- believe(P1,kill(P2,r,P1)).

% Permissibility and impermissibility for deontologism
impermissible(deontologism,P1):- violated(R,P1), maxim(R), beliefbase(P1).
permissible(deontologism,P1):- not impermissible(deontologism,P1), beliefbase(P1).


%%%%%%%%%%%%%%%  PRINCIPIALISM VERSION 1 %%%%%%%%%%%%%%%

% Principle prohibiting lying with exception and murder for principialism1
principle(doNotLieExcept;doNotEnableMurder).

% Rule stating violation of lying principle in the absence of first exception
violated(doNotLieExcept,P1) :- act(tell(P1,F,P2)), not believe(P1,F), not exception(act(tell(P1,F,P2)),doNotLieExcept).

% Rule stating that Falangists do not deserve truth
dontDeserveTruth(P2):- believe(p(P),murderer(P2)).

% First exception rule for lies: lying is permitted if truth is not deserved
exception(act(tell(P1,F,P2)),doNotLieExcept):- act(tell(P1,F,P2)), dontDeserveTruth(P2).

% Permissibility and impermissibility for principialism1
impermissible(principialism1,P1) :- violated(R,P1), principle(R).
permissible(principialism1,P1) :- not impermissible(principialism1,P1), beliefbase(P1).


%%%%%%%%%%%%%%%  UTILITIES CALCULATIONS %%%%%%%%%%%%%%%%%%

% Expected utilities for various physical actions according to consequentialism
uti(kill(P2,r,P1),-4):-person(P2), person(P1).
uti(harm1(P1,P2),-2):-person(P2), person(P1).
uti(harm2(P1,P2),-4):-person(P2), person(P1).
uti(wrongInfo(P1,P2),0):-person(P2), person(P1).
uti(help(P),2):-person(P).

% Actions triggering specific utility values
trigUti(Y,F,N) :- uti(F,N), believe(Y,F), beliefbase(Y).

% Calculation of total utility values for agents and actions
totalUti(Y,T):- T=#sum{N:uti(F,N),believe(Y,F)}, beliefbase(Y).


%#show act/1.
#show believe/2.
%#show impermissible/2.
%#show permissible/2.
%#show violated/2.
%#show trigUti/3.
%#show totalUti/2.
%#show objective_lie/2.
%#show objective_truth/2.
%#show erroneous_lie/2.
%#show erroneous_truth/2.
#show beliefbase/1.
%#show principle/1.
#show uti/2.
"""


In [ ]:
answers = ASP(program1)

def imprimer(N, L):
    print("\nAnswer Set ", N)
    for X in L:
        print(X[0], X[1])
n = 1
for answer in answers:
    imprimer(n, answer)
    n = n+1



Answer Set  1
uti ('harm2(p,f)', -4)
uti ('kill(p,r,f)', -4)
beliefbase ('env',)
uti ('wrongInfo(p,f)', 0)
beliefbase ('p(0)',)
believe ('p(0)', 'hostile(f)')
believe ('env', 'hostile(f)')
believe ('p(1)', 'ask(f,where,p,0)')
uti ('wrongInfo(f,p)', 0)
believe ('p(0)', 'credible(at(r,madrid),f)')
believe ('env', 'at(r,cemetery)')
uti ('harm2(f,p)', -4)
believe ('p(1)', 'hostile(f)')
uti ('help(p)', 2)
believe ('p(1)', 'credible(at(r,madrid),f)')
believe ('p(1)', 'credible(at(r,family),f)')
believe ('p(1)', 'murderer(f)')
uti ('kill(p,r,p)', -4)
uti ('kill(f,r,f)', -4)
uti ('harm1(p,f)', -2)
uti ('harm1(f,f)', -2)
uti ('wrongInfo(p,p)', 0)
uti ('harm1(f,p)', -2)
uti ('harm2(f,f)', -4)
uti ('kill(f,r,p)', -4)
believe ('p(0)', 'at(r,family)')
uti ('wrongInfo(f,f)', 0)
believe ('env', 'ask(f,where,p,0)')
uti ('harm2(p,p)', -4)
believe ('p(0)', 'ask(f,where,p,0)')
believe ('p(0)', 'credible(at(r,cemetery),f)')
uti ('help(f)', 2)
believe ('env', 'credible(at(r,cemetery),f)')
believe ('p(1)',

In [ ]:
#############  PRINCIPIALISM VERSION 2 ####################  +  #############  CONSEQUENTIALISM VERSIONS 1 AND 2 ####################


# Define program2
program2 = """


% PRINCIPIALISM VERSION 2
% Rule for comparing decisions based on counterfactual comparison
counterfactual(S1, S2, tell(P1, at(r, L1), P2)) :-
    act(S1,tell(P1, at(r, L1), P2)), L1!=L2,
    believe(P1, at(r, L2)),
    act(S2,tell(P1, at(r, L2), P2)).

% Second exception rule for lies: lying is permitted if truth leads to worst consequences
exceptionRel(S, doNotLieExcept) :-
    act(S,tell(p, at(r, L1), P2)),
    counterfactual(S, S2, tell(p, at(r, L1), P2)),
    totalUti(S, p, T1),
    totalUti(S2, p, T2),
    T1 > T2.

% Rule stating (local) violation of lying principle in the absence of second exception
violated(S, R) :-
    locally_violated(S, R),
    not exceptionRel(S, R).

% Permissibility and impermissibility for principialism2
permissible(principialism2, S) :- not impermissible(principialism2, S), act(S, _).
impermissible(principialism2, S) :- violated(S, R), principle(R).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% CONSEQUENTIALISM VERSION 1
% Permissibility and impermissibility for consequentialism1
impermissible(consequentialism1,S) :- totalUti(S,p,N1), totalUti(S2,p,N2), N2>N1.
permissible(consequentialism1,S) :- not impermissible(consequentialism1,S), act(S,_).


% CONSEQUENTIALISM VERSION 2
% Calculation of total utility based on sums of total utilities of each action
totalUti2(S,Y,T) :- T=#sum{N:totalUti(S,Y,N)}, beliefbase(Y),act(S,_).


% Permissibility and impermissibility for consequentialism2
impermissible(consequentialism2,S) :- totalUti2(S,env,N1), totalUti2(S2,env,N2), N2>N1.
permissible(consequentialism2,S) :- not impermissible(consequentialism2,S), act(S,_).

%#show totalUti/4.
%#show locally_violated/2.
%#show violated/2.
#show act/2.
%#show exceptionRel/2.
%#show counterfactual/3.
#show permissible/2.
#show impermissible/2.
"""

# Function to convert answer sets to valid ASP facts
def answer_set_to_facts(answer_set,n,first):
    formatted_facts = []
    transmit= [("believe_init",2),("believe_cons",2),("beliefbase",1), ("principle",1)]
    augment = [("act",1),("totalUti",2)]
    renamedAug = {("violated",1):"locally_violated"}
    for fact in answer_set:
        # Handle complex terms
        name = fact[0]
        args = fact[1] if isinstance(fact[1], tuple) else (fact[1],)
        arity = len(args)
        if first and (name,arity) in transmit :
          formatted_facts.append(f"{name}({','.join(map(str, args))}).")
        if (name,arity) in augment:
          newargs = ('s'+str(n),)+args
          formatted_facts.append(f"{name}({','.join(map(str, newargs))}).")
        if (name,arity) in renamedAug:
          newargs = ('s'+str(n),)+args
          newname = renamedAug[(name,arity)]
          formatted_facts.append(f"{newname}({','.join(map(str, newargs))}).")
    return "\n".join(formatted_facts)

# Run program1 to get its answer sets
answers_program1 = ASP(program1)
combined_program = ""

# Process each answer set from program1
for n, answer_set in enumerate(answers_program1, start=1):
    print(f"Processing Answer Set {n} of program1...")

    # Convert answer set to ASP facts
    facts_from_program1 = answer_set_to_facts(answer_set,n,(n==1))

    # Combine program1 and program2
    combined_program = combined_program+"%AS"+str(n)+"\n"+f"{facts_from_program1}\n"

combined_program=combined_program+"\n"+program2
print(combined_program)

Processing Answer Set 1 of program1...
%AS1
beliefbase(env).
totalUti(s1,p(1),0).
beliefbase(p(0)).
totalUti(s1,p(0),0).
totalUti(s1,env,0).
principle(doNotLieExcept).
principle(doNotEnableMurder).
beliefbase(p(1)).




% PRINCIPIALISM VERSION 2
% Rule for comparing decisions based on counterfactual comparison
counterfactual(S1, S2, tell(P1, at(r, L1), P2)) :-
    act(S1,tell(P1, at(r, L1), P2)), L1!=L2,
    believe(P1, at(r, L2)),
    act(S2,tell(P1, at(r, L2), P2)).

% Second exception rule for lies: lying is permitted if truth leads to worst consequences
exceptionRel(S, doNotLieExcept) :-
    act(S,tell(p, at(r, L1), P2)),
    counterfactual(S, S2, tell(p, at(r, L1), P2)),
    totalUti(S, p, T1),
    totalUti(S2, p, T2),
    T1 > T2.

% Rule stating (local) violation of lying principle in the absence of second exception
violated(S, R) :-
    locally_violated(S, R),
    not exceptionRel(S, R).

% Permissibility and impermissibility for principialism2
permissible(principialism2, S) :-

In [ ]:
# Run the combined program
answers_program2 = ASP(combined_program)

# Output results for this specific answer set
for m, result in enumerate(answers_program2, start=1):
        print(f"Answer Set {m} of program2 for Answer Set {n}:")
        for fact in result:
            print(fact)

Answer Set 1 of program2 for Answer Set 1:


## Résultats

| Act |  Deontologism|Principialism 1 | Principialism 2 | Consequentialism 1 | Consequentialism 2|
| :---------------: |:---------------:| :-----:|:-----:|:-----:|:-----:|
| tell madrid |Imp|Perm|Perm|Perm|Perm|
| tell family |Imp|Imp|Imp|Imp|Imp|
| tell cimetery  | Imp|Perm|Perm|Imp|Imp|
| silence  | Perm|Perm|Perm|Imp|Imp|